### Daily Journal Jupyter Notebook Code

This is a Jupyter Notebook with code copied directly from Moshe Zadka's article on opensource.com about writing daily journals in Jupyter Notebook.

In [1]:
pip install twisted

     |████████████████████████████████| 3.1 MB 2.1 MB/s eta 0:00:01
  Using cached incremental-17.5.0-py2.py3-none-any.whl (16 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl (7.9 kB)
  Using cached Automat-20.2.0-py2.py3-none-any.whl (31 kB)
     |████████████████████████████████| 193 kB 20.2 MB/s eta 0:00:01
  Using cached PyHamcrest-2.0.2-py3-none-any.whl (52 kB)
  Using cached hyperlink-20.0.1-py2.py3-none-any.whl (48 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import twisted.internet.asyncioreactor
twisted.internet.asyncioreactor.install()
from twisted.internet import reactor, task
import ipywidgets, datetime, subprocess, functools, os

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [3]:
class DoneError(Exception):
    pass

In [4]:
def time_out_counter(reactor):
    label = ipywidgets.Label("Time left: 10:00")
    current_seconds = datetime.timedelta(minutes=10).total_seconds()
    def decrement(count):
        nonlocal current_seconds
        current_seconds -= count
        time_left = datetime.timedelta(seconds=max(current_seconds, 0))
        minutes, left = divmod(time_left, minute)
        seconds = int(left.total_seconds())
        label.value = f"Time left: {minutes}:{seconds:02}"
        if current_seconds < 0:
            raise DoneError("finished")
    minute = datetime.timedelta(minutes=1)
    call = task.LoopingCall.withCount(decrement)
    call.reactor = reactor
    d = call.start(1)
    d.addErrback(lambda f: f.trap(DoneError))
    return d, label

In [5]:
def editor(fname):
    textarea = ipywidgets.Textarea(continuous_update=False)
    textarea.rows = 20
    output = ipywidgets.Output()
    runner = functools.partial(subprocess.run, capture_output=True, text=True, check=True)
    def save(_ignored):
        with output:
            with open(fname, "w") as fpout:
                fpout.write(textarea.value)
            print("Sending...", end='')
            try:
                runner(["git", "add", fname])
                runner(["git", "commit", "-m", f"updated {fname}"])
                runner(["git", "push"])
            except subprocess.CalledProcessError as exc:
                print("Could not send")
                print(exc.stdout)
                print(exc.stderr)
            else:
                 print("Done")
    textarea.observe(save, names="value")
    return textarea, output, save

In [6]:
def journal():
    date = str(datetime.date.today())
    title = f"Log: Startdate {date}"
    filename = os.path.join(f"{date}.txt")
    d, clock = time_out_counter(reactor)
    textarea, output, save = editor(filename)
    box = ipywidgets.VBox([
        ipywidgets.Label(title),
        textarea,
        clock,
        output
    ])
    d.addCallback(save)
    return box

In [7]:
journal()